# Multiclass Text Classification

Pada latihan kali ini kita akan melakukan klasifikasi teks multikelas menggunakan lstm.

Pada latihan ini kita akan menggunakan dataset yang berisi sinopsis dari beberapa film Indonesia dan genrenya. Tujuan kita adalah menentukan genre sebuah film berdasarkan sinopsisnya. Dataset dapat Anda unduh pada [tautan](https://www.kaggle.com/antoniuscs/imdb-synopsis-indonesian-movies) berikut.

Pada cell pertama impor library pandas dan ubah dataset menjadi dataframe. Kemudian buang kolom 'judul_film' karena kita hanya akan menggunakan sinopsis sebagai atribut untuk dilatih pada model.

In [ ]:
    import pandas as pd
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dicoding/dataset/imdb_indonesian_movies_2.csv')
    df = df.drop(columns=['judul_film'])

In [ ]:
df

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama
...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Proyek baru ini adalah tentang seorang lelaki ...,Romantis


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding. Jalankan kode di bawah untuk melakukan one-hot-encoding dan membuat dataframe baru.

In [ ]:
    category = pd.get_dummies(df.genre)
    df_baru = pd.concat([df, category], axis=1)
    df_baru = df_baru.drop(columns='genre')
    df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [ ]:
    sinopsis = df_baru['ringkasan_sinopsis'].values
    label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [ ]:
sinopsis

array(['Raden Mas Said putra sulung Tumenggung Wilarikta di bawah Kerajaan Majapahit yang berkuasa di wilayah Tuban melihat sekeluarga miskin yang menderita busung lapar. Ia merasa sangat prihatin dan hati nuraninya tergugah untuk menolong. Kemudian ia mencoba secara diam-diam mengambil makanan dari lumbung orang tuanya. Perbuatan itu tidak disetujui orang tuanya bahkan ia dihukum sekap di gudang makanan itu. Sejak kejadian itu RM Said yang tumbuh dewasa tidak betah tinggal di rumah. Ia berkelana dari daerah satu ke daerah lainnya. Dari sanalah ia tahu betapa banyak penyelewengan dan kesewenang-wenangan para lurah yang munafik. Mereka selalu mengkambing-hitamkan Tumenggung untuk menutupi kejahatannya. Atas laporan RM Said ayahnya kemudian sadar. Tetapi kemudian ia dianggap sebagai sumber fitnah. Dalam kelananya kemudian ia bertemu dengan Sunan Bonang yang banyak mencurahkan ilmunya kepada RM Said. Ia pun kemudian melakukan tapa di pinggir kali. Berkat ketabahannya menghadapi berbagai c

In [ ]:
label

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]], dtype=uint8)

Lalu, bagi data untuk training dan data untuk testing.

In [ ]:
    from sklearn.model_selection import train_test_split
    sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

Kemudian kita ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer. Setelah tokenisasi selesai, kita perlu membuat mengonversi setiap sampel menjadi sequence.

In [ ]:
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
     
    tokenizer = Tokenizer(num_words=5000, oov_token='x')
    tokenizer.fit_on_texts(sinopsis_latih) 
    tokenizer.fit_on_texts(sinopsis_test)
     
    sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
    sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
     
    padded_latih = pad_sequences(sekuens_latih) 
    padded_test = pad_sequences(sekuens_test)

In [ ]:
padded_latih

array([[   0,    0,    0, ..., 4678,   13, 4678],
       [   0,    0,    0, ...,  263,  355,  728],
       [   0,    0,    0, ...,   32,   22,  318],
       ...,
       [   0,    0,    0, ...,  141, 4678, 4678],
       [   0,    0,    0, ..., 4129,   75, 4678],
       [   0,    0,    0, ...,   60,  276,  211]], dtype=int32)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. Jangan lupa panggil fungsi compile dan tentukan optimizer serta loss function yang akan dipakai oleh model.

In [ ]:
    import tensorflow as tf
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Terakhir kita dapat mulai melatih model kita dengan memanggil fungsi fit().

In [ ]:
    num_epochs = 30
    history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                        validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 15s - loss: 1.6099 - accuracy: 0.1841 - val_loss: 1.6114 - val_accuracy: 0.1592
Epoch 2/30
26/26 - 1s - loss: 1.6057 - accuracy: 0.2114 - val_loss: 1.6145 - val_accuracy: 0.1642
Epoch 3/30
26/26 - 1s - loss: 1.5768 - accuracy: 0.3197 - val_loss: 1.6095 - val_accuracy: 0.2537
Epoch 4/30
26/26 - 1s - loss: 1.3421 - accuracy: 0.5000 - val_loss: 1.6969 - val_accuracy: 0.2537
Epoch 5/30
26/26 - 1s - loss: 0.8888 - accuracy: 0.5833 - val_loss: 2.0037 - val_accuracy: 0.3184
Epoch 6/30
26/26 - 1s - loss: 0.5103 - accuracy: 0.8259 - val_loss: 2.7253 - val_accuracy: 0.3134
Epoch 7/30
26/26 - 1s - loss: 0.2684 - accuracy: 0.9117 - val_loss: 2.8050 - val_accuracy: 0.2985
Epoch 8/30
26/26 - 1s - loss: 0.1483 - accuracy: 0.9590 - val_loss: 3.0948 - val_accuracy: 0.2836
Epoch 9/30
26/26 - 1s - loss: 0.0428 - accuracy: 0.9925 - val_loss: 3.7368 - val_accuracy: 0.3085
Epoch 10/30
26/26 - 1s - loss: 0.0156 - accuracy: 0.9988 - val_loss: 3.9989 - val_accuracy: 0.3035
Epoch 11/30
26/26 

Akurasi dari model kita menunjukkan terjadinya overfitting karena akurasi pada data testing sangat besar, sedangkan akurasi pada data validasi jauh lebih kecil. Hal ini masih sangat luar biasa karena kita hanya memiliki 1000 buah sampel data!